In [1]:
!pip install pysolar

     |████████████████████████████████| 47 kB 2.3 MB/s 


In [4]:
import os
os.chdir('/content/drive/Shareddrives/yujung kim/Study/Python/time_series')

In [21]:
import pandas as pd
from datetime import datetime, date, timezone, timedelta

In [9]:
pv = pd.read_csv('data/PV_hour.csv')
pv.columns = ['time', 'power']

# datetime

In [23]:
# datetime 으로 변환
tz = timezone(timedelta(hours=-6))
pv['datetime'] = [datetime(int(f"20{i[6:8]}"), int(i[:2]), int(i[3:5]), int(i[9:11]), tzinfo=tz) for i in pv['time']]
pv['month'] = [i.month for i in pv['datetime']]
pv['day'] = [i.day for i in pv['datetime']]

# solar info

In [33]:
from pysolar.solar import get_altitude, get_azimuth
from pysolar.radiation import get_radiation_direct

In [35]:
pv['alt'] = [get_altitude(latitude_deg=30.45, longitude_deg=-88.25, when=i) for i in pv['datetime']]
pv['azi'] = [get_azimuth(latitude_deg=30.45, longitude_deg=-88.25, when=i) for i in pv['datetime']]
pv['rad'] = [get_radiation_direct(when=pv['datetime'].iloc[i], altitude_deg=pv['alt'].iloc[i]) for i in range(pv.shape[0])]

/usr/local/lib/python3.7/dist-packages/pysolar/radiation.py:50: RuntimeWarning: overflow encountered in exp
  return flux * math.exp(-1 * optical_depth * air_mass_ratio) * is_daytime
/usr/local/lib/python3.7/dist-packages/pysolar/radiation.py:50: RuntimeWarning: invalid value encountered in multiply
  return flux * math.exp(-1 * optical_depth * air_mass_ratio) * is_daytime


In [36]:
pv.iloc[:10]

,time,power,datetime,month,day,alt,azi,rad
0,01/01/06 00,0.0,2006-01-01 00:00:00-06:00,1,1,-82.519778,6.327078,0.000000
1,01/01/06 01,0.0,2006-01-01 01:00:00-06:00,1,1,-74.001270,66.094544,0.000000
2,01/01/06 02,0.0,2006-01-01 02:00:00-06:00,1,1,-61.530551,82.353403,0.000000
3,01/01/06 03,0.0,2006-01-01 03:00:00-06:00,1,1,-48.633108,91.048671,0.000000
4,01/01/06 04,0.0,2006-01-01 04:00:00-06:00,1,1,-35.752419,97.771663,0.000000
5,01/01/06 05,0.0,2006-01-01 05:00:00-06:00,1,1,-23.063958,104.017664,0.000000
6,01/01/06 06,0.0,2006-01-01 06:00:00-06:00,1,1,-10.720856,110.478073,0.000000
7,01/01/06 07,30.2,2006-01-01 07:00:00-06:00,1,1,1.432672,117.660871,4.696817
8,01/01/06 08,58.0,2006-01-01 08:00:00-06:00,1,1,12.143450,126.079567,636.876159
9,01/01/06 09,139.3,2006-01-01 09:00:00-06:00,1,1,21.852863,136.311614,849.424805


# weather info

In [40]:
!pip install meteostat

In [60]:
from meteostat import Point, Daily, Hourly

In [61]:
alabama = Point(30.45, -88.25, 0)
start = datetime(2006, 1, 1)
end = datetime(2006, 12, 31, 23, 59)

# data = Daily(alabama, start, end)
data = Hourly(alabama, start, end)

In [78]:
pv_weather = data.fetch().reset_index(drop=True)
weather_columns = list(pv_weather.columns)
pv.loc[:,weather_columns] = pv_weather[weather_columns]

In [83]:
pv

,time,power,datetime,month,day,alt,azi,rad,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,01/01/06 00,0.0,2006-01-01 00:00:00-06:00,1,1,-82.519778,6.327078,0.0,17.6,15.9,90.0,NaN,NaN,130.0,5.4,NaN,1013.9,NaN,NaN
1,01/01/06 01,0.0,2006-01-01 01:00:00-06:00,1,1,-74.001270,66.094544,0.0,15.1,14.5,96.0,0.0,NaN,180.0,11.2,NaN,1014.7,NaN,NaN
2,01/01/06 02,0.0,2006-01-01 02:00:00-06:00,1,1,-61.530551,82.353403,0.0,15.1,14.5,96.0,0.0,NaN,200.0,11.2,NaN,1015.0,NaN,NaN
3,01/01/06 03,0.0,2006-01-01 03:00:00-06:00,1,1,-48.633108,91.048671,0.0,15.7,15.1,96.0,0.0,NaN,180.0,11.2,NaN,1014.9,NaN,NaN
4,01/01/06 04,0.0,2006-01-01 04:00:00-06:00,1,1,-35.752419,97.771663,0.0,15.7,15.1,96.0,0.0,NaN,170.0,9.4,NaN,1014.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12/31/06 19,0.0,2006-12-31 19:00:00-06:00,12,31,-24.605456,256.686082,0.0,19.5,14.5,73.0,0.0,NaN,340.0,9.4,NaN,1017.2,NaN,NaN
8756,12/31/06 20,0.0,2006-12-31 20:00:00-06:00,12,31,-37.320299,262.945079,0.0,20.1,13.3,65.0,0.0,NaN,310.0,7.6,NaN,1016.9,NaN,NaN
8757,12/31/06 21,0.0,2006-12-31 21:00:00-06:00,12,31,-50.210673,269.794216,0.0,20.7,12.9,61.0,0.0,NaN,310.0,0.0,NaN,1017.0,NaN,NaN
8758,12/31/06 22,0.0,2006-12-31 22:00:00-06:00,12,31,-63.090341,278.930730,0.0,18.4,14.1,76.0,0.0,NaN,290.0,0.0,NaN,1017.3,NaN,NaN


In [84]:
import pickle
with open('data/pv.pickle', 'wb') as f:
    pickle.dump(pv, f)